<a href="https://colab.research.google.com/github/KarunakarMuppuri/INFO-5731-Section-020/blob/Info-assigmnet-2/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [ ]:
#The headers for amazon wer scrape expires after sometime if DOM doesnt return any HTML structure 
# Please use new headers and try again. I'm  also gonna attach a copy of csv file in canvas just in case
# Headers are expired.
import requests
from bs4 import BeautifulSoup
import pandas as pd

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers",
}
URL = "https://www.amazon.com/product-reviews/B08BX7FV5L/ref=zg-bs_1232597011_cr_sccl_4/142-0881380-3195946?pd_rd_w=pMDOb&content-id=amzn1.sym.7cc5d8c9-0275-49fe-af63-f984f7dc4330&pf_rd_p=7cc5d8c9-0275-49fe-af63-f984f7dc4330&pf_rd_r=W8BJ2R4R18FT61DQCVFD&pd_rd_wg=VLy5Z&pd_rd_r=fa728cfe-04f0-43ca-a6e9-13ad54ea26df&pd_rd_i=B08BX7FV5L"

def scrape_reviews(url: str) -> pd.DataFrame:
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    print(soup)
    reviews = soup.find_all("div", {"data-hook": "review"})
    data = []

    for review in reviews:
        title = review.find("a", {"data-hook": "review-title"}).text.strip()
        rating = review.find("i", {"data-hook": "review-star-rating"}).text.strip()
        text = review.find("span", {"data-hook": "review-body"}).text.strip()
        data.append([title, rating, text])

    df = pd.DataFrame(data, columns=["Title", "Rating", "Review Text"])
    return df

if __name__ == "__main__":
    df = scrape_reviews(URL)
    df.to_csv("product_reviews.csv", index=False)


<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER"

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import pandas as pd
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv("product_reviews.csv")

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    stopwords_list = stopwords.words('english')
    stopwords_list.remove('not')
    tokens = [word for word in tokens if word not in stopwords_list]
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
    cleaned_text = " ".join(lemmatized_tokens)
    return cleaned_text

df['Cleaned Text'] = df['Review Text'].apply(clean_text)

df.to_csv("product_reviews.csv", index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import csv
import spacy

nlp = spacy.load('en_core_web_sm')

def pos_tagging(text):
    doc = nlp(text)
    pos = []
    for token in doc:
        pos.append((token.text, token.pos_))
    pos_count = dict(zip(set(pos), [pos.count(i) for i in set(pos)]))
    return pos, pos_count

def constituency_parsing(text):
    doc = nlp(text)
    return list(doc.sents)[0].root.lefts

def dependency_parsing(text):
    doc = nlp(text)
    return list(doc.sents)[0].root.children

def entity_recognition(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ not in ['OTHER', 'CARDINAL']:
            entities.append((ent.text, ent.label_))
    entity_count = {}
    for entity in entities:
        if entity[0] not in entity_count:
            entity_count[entity[0]] = 1
        else:
            entity_count[entity[0]] += 1
    entity_count_str = [f"{entity}: {count}" for entity, count in entity_count.items()]
    return entities, entity_count_str

with open('product_reviews.csv', 'r') as file:
    reader = csv.DictReader(file)
    rows = list(reader)

with open('product_reviews.csv', 'w', newline='') as file:
    fieldnames = ['Title', 'Review Text', 'Cleaned Text', 'POS', 'POS Count', 'Constituency Parsing', 'Dependency Parsing', 'Entities', 'Entity Count']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for row in rows:
        pos, pos_count = pos_tagging(row['Cleaned Text'])
        constituency_parse = [str(child) for child in constituency_parsing(row['Cleaned Text'])]
        dependency_parse = [str(child) for child in dependency_parsing(row['Cleaned Text'])]
        entity_count = entity_recognition(row['Cleaned Text'])

        writer.writerow({
            'Title': row['Title'],
            'Review Text': row['Review Text'],
            'Cleaned Text': row['Cleaned Text'],
            'POS': pos,
            'POS Count': pos_count,
            'Constituency Parsing': constituency_parse,
            'Dependency Parsing': dependency_parse,
            'Entities': entity_count[0],
            'Entity Count': entity_count[1]
        })


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency parsing involves breaking down a sentence into its phrases and creating a tree structure that shows the grammatical relationships between them. The root of the tree represents the whole sentence, and each node represents a phrase. For example, "The cat in the hat sat on the mat" can be broken down into a tree with branches representing the noun phrase and verb phrase. The noun phrase further breaks down into a determiner and a noun, while the verb phrase consists of a prepositional phrase and a noun phrase.

On the other hand, dependency parsing involves identifying the grammatical relationships between the words in a sentence and creating a graph that represents these relationships. Each word is a node in the graph, and the relationships between them are shown by directed edges. For instance, in "The cat in the hat sat on the mat," the main verb "sat" is the root of the tree, and the other words are connected to it according to their grammatical roles. "Cat" is connected to "sat" as its subject, while "on" is connected to "sat" as the preposition introducing the prepositional phrase "on the mat." The edges are labeled with the grammatical relationship between the words.